In [1]:
!pip -q install flask faiss-cpu sentence-transformers transformers torch pdfplumber pyngrok


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 49.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 2.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 49.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 46.1 MB/s eta 0:00:00


In [ ]:
!pip install -q faiss-cpu


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.8/23.8 MB 67.5 MB/s eta 0:00:00


In [ ]:
!pip install -q flask pyngrok sentence-transformers transformers


In [2]:
import faiss
import pickle

index = faiss.read_index("cuad_faiss.index")

with open("all_chunks.pkl", "rb") as f:
    all_chunks = pickle.load(f)

print("FAISS + chunks loaded")


RuntimeError: Error in faiss::FileIOReader::FileIOReader(const char*) at /project/third-party/faiss/faiss/impl/io.cpp:69: Error: 'f' failed: could not open cuad_faiss.index for reading: No such file or directory

In [38]:
import torch
from sentence_transformers import SentenceTransformer
from transformers import AutoTokenizer, AutoModelForCausalLM

device = "cuda" if torch.cuda.is_available() else "cpu"

embed_model = SentenceTransformer("all-MiniLM-L6-v2")

tokenizer = AutoTokenizer.from_pretrained("gpt2")
tokenizer.pad_token = tokenizer.eos_token

model = AutoModelForCausalLM.from_pretrained("gpt2").to(device)


In [39]:
def retrieve_chunks(query, top_k=5):
    q_emb = embed_model.encode(
        [query],
        convert_to_numpy=True,
        normalize_embeddings=True
    )
    scores, idx = index.search(q_emb, top_k)
    return [all_chunks[i] for i in idx[0]]


In [40]:
def analyze_vendor_risks(chunks):
    risks = {}

    rules = {
        "Indemnification Risk": {
            "keywords": ["indemnify", "indemnification"],
            "severity": "High",
            "summary": "Indemnification obligations may expose the vendor to long-term liability."
        },
        "Liability & Damages Risk": {
            "keywords": ["limitation of liability", "liable", "consequential", "punitive"],
            "severity": "High",
            "summary": "Liability limitations and damage exclusions may significantly shift risk."
        },
        "Termination Risk": {
            "keywords": ["termination", "terminate"],
            "severity": "Medium",
            "summary": "Certain obligations survive termination, increasing post-contract exposure."
        }
    }

    for chunk in chunks:
        lower = chunk.lower()
        for risk, rule in rules.items():
            if any(k in lower for k in rule["keywords"]):
                if risk not in risks:
                    risks[risk] = {
                        "risk": risk,
                        "severity": rule["severity"],
                        "summary": rule["summary"],
                        "evidence": chunk.strip().split(".")[0] + "."
                    }

    return list(risks.values())


In [41]:
def generate_answer(query, top_k=5):
    # 1️⃣ Retrieve relevant chunks using FAISS
    chunks = retrieve_chunks(query, top_k)

    # 2️⃣ RISK MODE (your code goes HERE 👇)
    if "risk" in query.lower():
        risks = analyze_vendor_risks(chunks)

        if not risks:
            return "No material vendor risks identified in this contract.", []

        answer = "⚠️ Identified Contract Risks\n\n"

        for i, r in enumerate(risks, 1):
            answer += (
                f"{i}️⃣ {r['risk']} ({r['severity']})\n"
                f"• {r['summary']}\n"
                f"• Evidence: {r['evidence']}\n\n"
            )

        # ⛔ Important: do NOT return chunks again
        return answer, []

    # 3️⃣ NORMAL QUESTION-ANSWER MODE
    context = "\n\n".join(chunks)

    prompt = f"""
You are a legal assistant.
Answer the question using ONLY the context below.
If the answer is not present, say "Not found in contract."

Context:
{context}

Question:
{query}

Answer:
"""

    inputs = tokenizer(
        prompt,
        return_tensors="pt",
        truncation=True,
        max_length=1024
    ).to(device)

    outputs = model.generate(
        **inputs,
        max_new_tokens=250,
        temperature=0.2,
        do_sample=False,
        pad_token_id=tokenizer.eos_token_id
    )

    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)

    if "Answer:" in decoded:
        decoded = decoded.split("Answer:")[-1].strip()

    return decoded, chunks


In [42]:
from flask import Flask, request, render_template

app = Flask(__name__)

@app.route("/", methods=["GET"])
def home():
    return render_template("index.html")


@app.route("/ask", methods=["POST"])
def ask():
    query = request.form.get("query")

    # 👇 CALL your RAG function
    answer, chunks = generate_answer(query)

    # 👇 UI decision logic (THIS is where your code belongs)
    show_chunks = False if "risk" in query.lower() else True

    return render_template(
        "index.html",
        answer=answer,
        chunks=chunks if show_chunks else None,
        query=query
    )


In [43]:
html = """<!DOCTYPE html>
<html>
<head>
    <title>Intelligent Contract Review Assistant</title>

    <style>
        body {
            font-family: Arial, sans-serif;
            margin: 40px;
            background-color: #fafafa;
        }

        h1 {
            color: #333;
        }

        input[type="text"] {
            width: 70%;
            padding: 12px;
            font-size: 14px;
        }

        button {
            padding: 12px 20px;
            font-size: 14px;
            background-color: #1976d2;
            color: white;
            border: none;
            cursor: pointer;
        }

        button:hover {
            background-color: #125aa3;
        }

        .answer-box {
            background-color: #fff8e1;
            border-left: 6px solid #ff9800;
            padding: 20px;
            margin-top: 30px;
        }

        .chunks-box {
            background-color: #f4f4f4;
            padding: 20px;
            margin-top: 20px;
            border-left: 6px solid #9e9e9e;
        }

        pre {
            white-space: pre-wrap;
            font-family: Arial, sans-serif;
            font-size: 14px;
        }

        hr {
            margin: 15px 0;
        }
    </style>
</head>

<body>

<h1>📄 Intelligent Contract Review Assistant (RAG)</h1>
<p>
    Ask questions like:
    <i>"termination conditions", "payment terms", "vendor risk"</i>
</p>

<form method="POST" action="/ask">
    <input
        type="text"
        name="query"
        placeholder="Ask a question about the contract..."
        value="{{ query or '' }}"
        required
    >
    <button type="submit">Ask</button>
</form>

{% if answer %}
<div class="answer-box">
    <h3>✅ Answer</h3>
    <pre>{{ answer }}</pre>
</div>
{% endif %}

{% if chunks %}
<div class="chunks-box">
    <h3>📌 Retrieved Contract Evidence</h3>
    {% for c in chunks %}
        <hr>
        <pre>{{ c }}</pre>
    {% endfor %}
</div>
{% endif %}

</body>
</html>
"""

with open("templates/index.html", "w") as f:
    f.write(html)


In [44]:
from pyngrok import ngrok

ngrok.set_auth_token("38z8e7xeNGIQnptGouLvTsDGMAt_7MYXfJP2ZJy2G4wLKf3Vc")


In [45]:
from threading import Thread
from pyngrok import ngrok

Thread(target=lambda: app.run(host="0.0.0.0", port=5000, debug=False, use_reloader=False)).start()

public_url = ngrok.connect(5000)
print("🌍 Public URL:", public_url)


 * Serving Flask app '__main__'
 * Debug mode: off


Address already in use
Port 5000 is in use by another program. Either identify and stop that program, or start the server with a different port.


🌍 Public URL: NgrokTunnel: "https://luis-meek-noriko.ngrok-free.dev" -> "http://localhost:5000"
